<a href="https://colab.research.google.com/github/dolmani38/similarly/blob/master/Text%20Generation%20With%20LSTM%20Recurrent%20Neural%20Networks%20in%20Python%20with%20Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Text Generation With LSTM Recurrent Neural Networks in Python with Keras

https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

In [2]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [30]:
# load ascii text and covert to lowercase
target_url = "https://www.gutenberg.org/files/11/11-0.txt"
import urllib.request
raw_text = urllib.request.urlopen(target_url).read().decode('utf-8')
raw_text = raw_text.lower()

In [31]:
raw_text


'\ufeffthe project gutenberg ebook of alice’s adventures in wonderland, by lewis carroll\r\n\r\nthis ebook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  you may copy it, give it away or\r\nre-use it under the terms of the project gutenberg license included\r\nwith this ebook or online at www.gutenberg.org\r\n\r\n\r\ntitle: alice’s adventures in wonderland\r\n\r\nauthor: lewis carroll\r\n\r\nrelease date: june 25, 2008 [ebook #11]\r\nlast updated: february 22, 2020\r\n\r\nlanguage: english\r\n\r\ncharacter set encoding: utf-8\r\n\r\n*** start of this project gutenberg ebook alice’s adventures in wonderland ***\r\n\r\n\r\n\r\nproduced by arthur dibianca and david widger\r\n\r\n[illustration]\r\n\r\n\r\n\r\n\r\nalice’s adventures in wonderland\r\n\r\nby lewis carroll\r\n\r\nthe millennium fulcrum edition 3.0\r\n\r\ncontents\r\n\r\n chapter i.     down the rabbit-hole\r\n chapter ii.    the pool of tears\r\n chapter iii.   a caucus-race and a 

In [32]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [33]:
chars

['\n',
 '\r',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 '@',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'ù',
 '—',
 '‘',
 '’',
 '“',
 '”',
 '\ufeff']

In [34]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  167975
Total Vocab:  66


In [35]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  167875


In [36]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [37]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 65)                16705     
Total params: 280,897
Trainable params: 280,897
Non-trainable params: 0
_________________________________________________________________


In [45]:
#Larger LSTM Recurrent Neural Network

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 65)                16705     
Total params: 806,209
Trainable params: 806,209
Non-trainable params: 0
_________________________________________________________________


In [46]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [47]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
1312/1312 [==============================] - ETA: 0s - loss: 2.9283
Epoch 00001: loss improved from inf to 2.92829, saving model to weights-improvement-01-2.9283.hdf5
1312/1312 [==============================] - 100s 77ms/step - loss: 2.9283
Epoch 2/20
1312/1312 [==============================] - ETA: 0s - loss: 2.6400
Epoch 00002: loss improved from 2.92829 to 2.63995, saving model to weights-improvement-02-2.6400.hdf5
1312/1312 [==============================] - 100s 76ms/step - loss: 2.6400
Epoch 3/20
1312/1312 [==============================] - ETA: 0s - loss: 2.4647
Epoch 00003: loss improved from 2.63995 to 2.46469, saving model to weights-improvement-03-2.4647.hdf5
1312/1312 [==============================] - 100s 76ms/step - loss: 2.4647
Epoch 4/20
1312/1312 [==============================] - ETA: 0s - loss: 2.3272
Epoch 00004: loss improved from 2.46469 to 2.32721, saving model to weights-improvement-04-2.3272.hdf5
1312/1312 [==============================] - 100s 7

In [48]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print(''.join([int_to_char[value] for value in pattern]))

Seed:
 derivative works based on the work as long as
all references to project gutenberg are removed. of 


In [51]:
import sys
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

  *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *      *  

KeyboardInterrupt: ignored